In [1]:
import sys, os, imp

ROOT_DIR = '/home/dileep/github_repos/mimic3-benchmarks'

from mimic3benchmark.readers import MultitaskReader
from mimic3models.preprocessing import Discretizer, Normalizer

args_dict = {
    'target_repl_coef': 0.0,
    'partition': 'custom',
    'ihm_C': 0.2,
    'los_C': 1.5,
    'pheno_C': 1.0,
    'decomp_C': 1.0,
    'data': '/home/dileep/github_repos/mimic3-benchmarks/data/multitask/train',
    'output_dir': '.',
    'mode': 'train',
    'timestep': 1,
    'network': 'mimic3models/keras_models/multitask_lstm.py',
    'dim': 512,
    'batch_size': 16,
    'dropout': 0.3
}

target_repl_coef = args_dict['target_repl_coef']
partition = args_dict['partition']
ihm_C = args_dict['ihm_C']
los_C = args_dict['los_C']
pheno_C = args_dict['pheno_C']
decomp_C = args_dict['decomp_C']
data = args_dict['data']
output_dir = args_dict['output_dir']
mode = args_dict['mode']
timestep = args_dict['timestep']

# Preparing reader
dataset_dir = os.path.join(ROOT_DIR, 'data/multitask/train')
listfile_dir = os.path.join(ROOT_DIR, 'data/multitask/train_listfile.csv')

reader = MultitaskReader(dataset_dir=dataset_dir, listfile=listfile_dir)

# Preparing discretizer
discretizer = Discretizer(timestep=timestep,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

discretizer_header = discretizer.transform(reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

# Preparing normalizer
normalizer = Normalizer(fields=cont_channels)

normalizer_state = os.path.join(ROOT_DIR, 'mimic3models/multitask/mult_ts1.0.input_str:previous.start_time:zero.normalizer')
normalizer.load_params(normalizer_state)

# Loading module
network = 'mimic3models/keras_models/multitask_lstm.py'
model_module = imp.load_source(os.path.basename(network), network)

target_repl = (target_repl_coef > 0.0 and mode == 'train')

args_dict['header'] = discretizer_header,
args_dict['ihm_pos'] = int(48.0/args_dict['timestep'] - 1e-6),
args_dict['target_repl'] = target_repl

Using TensorFlow backend.
/home/dileep/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dileep/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dileep/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dileep/anaconda3/env

In [3]:
model = model_module.Network(**args_dict)

TypeError: __init__() missing 2 required positional arguments: 'batch_norm' and 'rec_dropout'